In [2]:
# !wget https://huggingface.co/datasets/coseal/CodeUltraFeedback/resolve/main/data/train-00000-of-00001.parquet -O CodeUltraFeedback.parquet 

In [3]:
import pandas as pd

df = pd.read_parquet('CodeUltraFeedback.parquet')
df.shape

(10000, 5)

In [27]:
texts = []
texts.extend(df['instruction'].tolist())

for i in range(len(df)):
    for k in range(len(df['responses'][i])):
        texts.append(df['responses'][i][k]['response'])
        
    for k in range(len(df['annotations'][i])):
        texts.append(df['annotations'][i][k]['rationale'])

texts = list(set(texts))

In [29]:
import json

with open('CodeUltraFeedback.texts', 'w') as fopen:
    for t in texts:
        if not len(t):
            continue
        fopen.write(f'{json.dumps(t)}\n')

In [30]:
!wc -l CodeUltraFeedback.texts

89070 CodeUltraFeedback.texts


In [33]:
import malaya

model = malaya.translation.huggingface(
    model = 'mesolitica/translation-t5-small-standard-bahasa-cased-v2',
    force_check = False,
    use_ctranslate2 = True, 
    device = 'cuda', 
    quantization = 'bfloat16',
)

/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [38]:
max_length = 5120
to = 'ms'

In [46]:
from tqdm import tqdm

results = []
with open('CodeUltraFeedback.texts') as fopen:
    for l in tqdm(fopen):
        text = json.loads(l)
        r = model.generate([text], to_lang = to, max_input_length = max_length, max_decoding_length = max_length)[0]
        data = {'src': text, 'r': r}
        results.append(data)

89070it [12:23:53,  2.00it/s]


In [50]:
with open('CodeUltraFeedback.texts.translated', 'w') as fopen:
    json.dump(results, fopen)

In [93]:
filtered, accepted = [], {}
for r in results:
    try:
        splitted = r['r'].split()
        ratio = (len(set(splitted)) / len(splitted))
        if ratio < 0.5:
            filtered.append((r, ratio))
        else:
            accepted[r['src']] = r['r']
    except:
        filtered.append((r, 0))
        
len(accepted), len(filtered)

(80281, 8789)

In [116]:
row = df.iloc[0].to_dict()
row['responses'] = row['responses'].tolist()
row['annotations'] = row['annotations'].tolist()
row['instruction_ms'] = accepted.get(row['instruction'])

for i in range(len(row['responses'])):
    row['responses'][i]['response_ms'] = accepted.get(row['responses'][i]['response'])
    
for i in range(len(row['annotations'])):
    row['annotations'][i]['rationale_ms'] = accepted.get(row['annotations'][i]['rationale'])

In [120]:
with open('translated-CodeUltraFeedback.jsonl', 'w') as fopen:
    for k in range(len(df)):
        row = df.iloc[k].to_dict()
        row['responses'] = row['responses'].tolist()
        row['annotations'] = row['annotations'].tolist()
        row['models'] = row['models'].tolist()
        row['instruction_ms'] = accepted.get(row['instruction'])

        for i in range(len(row['responses'])):
            row['responses'][i]['response_ms'] = accepted.get(row['responses'][i]['response'])

        for i in range(len(row['annotations'])):
            row['annotations'][i]['rationale_ms'] = accepted.get(row['annotations'][i]['rationale'])
        
        fopen.write(f'{json.dumps(row)}\n')

In [121]:
from huggingface_hub import HfApi
api = HfApi()

In [122]:
api.upload_file(
    path_or_fileobj='CodeUltraFeedback.texts.translated',
    path_in_repo='CodeUltraFeedback.texts.translated',
    repo_id='mesolitica/translated-CodeUltraFeedback',
    repo_type='dataset',
)

CodeUltraFeedback.texts.translated:   0%|          | 0.00/201M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/translated-CodeUltraFeedback/commit/bdc431e231a71335b8440cafc4cd5b871f02541a', commit_message='Upload CodeUltraFeedback.texts.translated with huggingface_hub', commit_description='', oid='bdc431e231a71335b8440cafc4cd5b871f02541a', pr_url=None, pr_revision=None, pr_num=None)

In [123]:
api.upload_file(
    path_or_fileobj='translated-CodeUltraFeedback.jsonl',
    path_in_repo='translated-CodeUltraFeedback.jsonl',
    repo_id='mesolitica/translated-CodeUltraFeedback',
    repo_type='dataset',
)

translated-CodeUltraFeedback.jsonl:   0%|          | 0.00/172M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/translated-CodeUltraFeedback/commit/c238a1b20795afb9de5cfd8b9918dc73990d29fd', commit_message='Upload translated-CodeUltraFeedback.jsonl with huggingface_hub', commit_description='', oid='c238a1b20795afb9de5cfd8b9918dc73990d29fd', pr_url=None, pr_revision=None, pr_num=None)